<a href="https://colab.research.google.com/github/LuisCGit/mit_bitcoinexpo_hackathon/blob/main/NFT_Neural_Network_Embedding_withimage.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyyaml==5.4.1
! pip install sentence_transformers
! pip install img2vec_pytorch

In [36]:
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from img2vec_pytorch import Img2Vec
from PIL import Image
from scipy import spatial
import requests
import socket
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
socket.setdefaulttimeout(30) # 60 seconds

In [ ]:
def imagevec(i):
  print(i)
  if nftdf['url'][i][-3:]=='mov':
    url = 'https://ipfs.io/ipfs/'+nftdf['url'][0][7:]
  if nftdf['url'][i][7]!='i':
    url = 'https://ipfs.io/ipfs/'+nftdf['url'][i][7:]
  else:
    url = 'https://ipfs.io/ipfs/'+nftdf['url'][i][12:]
  try:
    urllib.request.urlretrieve(
      url,
      "gfg.png")
  except socket.timeout:
    return [0]*512
  except ConnectionResetError:
    imagevec(i)
    
  img = Image.open("gfg.png")
  im = Image.new('RGB', img.size, (255, 255, 255))
  im.paste(img, None)

  return img2vec.get_vec(im, tensor=False)

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
model = SentenceTransformer('bert-base-nli-mean-tokens')

Downloading:   0%|          | 0.00/391 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.95k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/625 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/399 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

In [5]:
#load nft dataset
nftdf = pd.read_csv('drive/MyDrive/mitbitcoin_hackathon/nfts_100.csv').drop(columns=['Unnamed: 0'])
nftdf = nftdf.drop(columns = ['tokenId','recipient'])
nftdf = nftdf.dropna().reset_index().drop(columns=['index'])

In [6]:
#name embeddings
name_embeddings = model.encode(nftdf['name'],normalize_embeddings = True, show_progress_bar=True,batch_size = 2*64)

Batches:   0%|          | 0/8 [00:00<?, ?it/s]

In [ ]:
#description embeddings
desc_embeddings = model.encode(nftdf['description'], normalize_embeddings=True, show_progress_bar=True, batch_size = 2*64)

In [ ]:
#attribute embeddings
att_embeddings = model.encode(nftdf['attributes'],normalize_embeddings = True, show_progress_bar=True, batch_size = 2*64)

Batches:   0%|          | 0/8 [00:00<?, ?it/s]

In [ ]:
#image embeddings
vec = []
for i in range(0,nftdf.shape[0]):
  vec.append(imagevec(i))
  if i%5==0:
    pd.DataFrame(vec).to_csv('drive/MyDrive/mitbitcoin_hackathon/image_emb.csv',index=False)

In [19]:
image_emb = pd.read_csv('drive/MyDrive/mitbitcoin_hackathon/image_emb.csv')
image_emb = pd.concat([nftdf.iloc[:-1,:],image_emb],axis=1)

In [25]:
vec = []
for i in range(0,image_emb.shape[0]):
  if image_emb.iloc[i,5]!=0:
    vec.append(image_emb.iloc[i,:])

In [28]:
meta_imageembdf = pd.DataFrame(vec).reset_index().drop(columns=['index'])

In [30]:
imageembdf = meta_imageembdf.iloc[:,5:]

In [ ]:
nftdf2 = pd.concat([pd.DataFrame(name_embeddings),pd.DataFrame(desc_embeddings),pd.DataFrame(att_embeddings)],axis=1)

In [ ]:
nftdf2.to_csv('nftdf2.csv',index=False)

In [ ]:
nftdf2 = pd.read_csv('nftdf2.csv')

In [ ]:
#pca = PCA(n_components = 3)
#nft_embed = pca.fit_transform(nftdf2)


#tsne 
nft_tsne = TSNE(n_components=3, n_jobs = -1).fit_transform(nftdf2)

In [ ]:
#for image
image_tsne = TSNE(n_components=3, n_jobs = -1).fit_transform(imageembdf)

In [38]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=6)

In [ ]:
#nft_labels = kmeans.fit_predict(nft_embed)
#nft_tsne_labels = kmeans.fit_predict(nft_tsne)
nft_tsne_labels = nftdf['contract']

In [39]:
#image
image_labels = kmeans.fit_predict(image_tsne)
imagetsnelabelsdf = pd.DataFrame(image_labels).rename(columns={0: "labels"})

In [ ]:
#labelsdf = pd.DataFrame(nft_labels).rename(columns={0: "labels"})
tsnelabelsdf = pd.DataFrame(nft_tsne_labels).rename(columns={0: "labels"})

In [ ]:
#pdf = pd.concat([pd.DataFrame(nft_embed),labelsdf],axis=1)
pdftsne = pd.concat([pd.DataFrame(nft_tsne),tsnelabelsdf],axis=1)

In [40]:
#pdfimagetsne
pdfimagetsne = pd.concat([pd.DataFrame(image_tsne),imagetsnelabelsdf],axis=1)

In [47]:
meta_imageembdf['contract']

0      SP1QK1AZ24R132C0D84EEQ8Y2JDHARDR58R72E1ZW.boom...
1      SP1QK1AZ24R132C0D84EEQ8Y2JDHARDR58R72E1ZW.boom...
2      SP1QK1AZ24R132C0D84EEQ8Y2JDHARDR58R72E1ZW.boom...
3      SP1QK1AZ24R132C0D84EEQ8Y2JDHARDR58R72E1ZW.boom...
4      SP1QK1AZ24R132C0D84EEQ8Y2JDHARDR58R72E1ZW.boom...
                             ...                        
710    SP3D0WY16NZXVTF8Q8849Y2R93GNWJ162RZF1D1VQ.poke...
711    SP3D0WY16NZXVTF8Q8849Y2R93GNWJ162RZF1D1VQ.poke...
712    SP3D0WY16NZXVTF8Q8849Y2R93GNWJ162RZF1D1VQ.poke...
713    SP3D0WY16NZXVTF8Q8849Y2R93GNWJ162RZF1D1VQ.poke...
714    SP3D0WY16NZXVTF8Q8849Y2R93GNWJ162RZF1D1VQ.poke...
Name: contract, Length: 715, dtype: object

In [ ]:
pdfimagetsne

In [ ]:
#image embeddings
fig = px.scatter_3d(
    pdfimagetsne, x=0, y=1, z=2,
    color=meta_imageembdf['contract'], 
    labels={'influence_rating': 'Legend'},
    hover_data = [meta_imageembdf['name']]
)
fig.show()

In [ ]:
vec = []
for i in range(0,nftdf.shape[0]):
  vec.append(imagevec(i))
  if i%5==0:
    pd.DataFrame(vec).to_csv('drive/MyDrive/mitbitcoin_hackathon/image_emb.csv',index=False)

In [ ]:
image_emb = pd.read_csv('drive/MyDrive/mitbitcoin_hackathon/image_emb.csv')

In [ ]:
vec = []
for i in range(0,image_emb.shape[0]):
  if image_emb['0'][i]!=0:
    vec.append(image_emb.iloc[i,:])

In [ ]:
imageembdf = pd.DataFrame(vec).reset_index().drop(columns=['index'])

In [ ]:

len(vec)

527

In [ ]:
pd.DataFrame(vec).to_csv('drive/MyDrive/mitbitcoin_hackathon/image_emb.csv',index=False)

In [ ]:
image_emb = pd.DataFrame(vec)

In [ ]:
image_emb

,0,1,2,3,4,5,6,7,8,9,...,502,503,504,505,506,507,508,509,510,511
0,1.312663,0.130007,2.039576,0.839653,0.696579,0.356162,0.770528,0.955934,1.247197,1.724646,...,0.557956,1.455683,1.515276,3.628532,0.970384,0.679406,2.431937,0.421824,1.091177,0.957912
1,1.312663,0.130007,2.039576,0.839653,0.696579,0.356162,0.770528,0.955934,1.247197,1.724646,...,0.557956,1.455683,1.515276,3.628532,0.970384,0.679406,2.431937,0.421824,1.091177,0.957912
2,1.312663,0.130007,2.039576,0.839653,0.696579,0.356162,0.770528,0.955934,1.247197,1.724646,...,0.557956,1.455683,1.515276,3.628532,0.970384,0.679406,2.431937,0.421824,1.091177,0.957912
3,1.312663,0.130007,2.039576,0.839653,0.696579,0.356162,0.770528,0.955934,1.247197,1.724646,...,0.557956,1.455683,1.515276,3.628532,0.970384,0.679406,2.431937,0.421824,1.091177,0.957912
4,1.312663,0.130007,2.039576,0.839653,0.696579,0.356162,0.770528,0.955934,1.247197,1.724646,...,0.557956,1.455683,1.515276,3.628532,0.970384,0.679406,2.431937,0.421824,1.091177,0.957912
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
522,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
523,1.312663,0.130007,2.039576,0.839653,0.696579,0.356162,0.770528,0.955934,1.247197,1.724646,...,0.557956,1.455683,1.515276,3.628532,0.970384,0.679406,2.431937,0.421824,1.091177,0.957912
524,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
525,1.312663,0.130007,2.039576,0.839653,0.696579,0.356162,0.770528,0.955934,1.247197,1.724646,...,0.557956,1.455683,1.515276,3.628532,0.970384,0.679406,2.431937,0.421824,1.091177,0.957912


In [ ]:
import urllib

In [ ]:
i=9
if nftdf['url'][i][-3:]=='mov':
  url = 'https://ipfs.io/ipfs/'+nftdf['url'][0][7:]
if nftdf['url'][i][7]!='i':
  url = 'https://ipfs.io/ipfs/'+nftdf['url'][i][7:]
else:
  url = 'https://ipfs.io/ipfs/'+nftdf['url'][i][12:]
try:
  urllib.request.urlretrieve(
    url,
    "gfg.png")
except socket.timeout:
  print([0]*512)
img = Image.open("gfg.png")

im = Image.new('RGB', img.size, (255, 255, 255))
im.paste(img, None)

temp = img2vec.get_vec(im, tensor=False)

In [ ]:

from joblib import Parallel, delayed

In [ ]:
vec = np.array((nftdf.shape[0],512))

In [ ]:
temp

In [ ]:
i=494
if nftdf['url'][i][-3:]=='mov':
  'https://ipfs.io/ipfs/'+nftdf['url'][0][7:]
if nftdf['url'][i][7]!='i':
  print('https://ipfs.io/ipfs/'+nftdf['url'][i][7:])
else:
  print('https://ipfs.io/ipfs/'+nftdf['url'][i][12:])

https://ipfs.io/ipfs/QmQc8cmBbWC82LoP24QRKYR6nVBphoGCk14WaqYHmer3kr/images/164.png


In [ ]:
nftdf['url'][33][-3:]

'gif'

In [ ]:
nftdf.shape[0]

932

In [ ]:
pd.DataFrame(vec)

,0,1,2,3,4,5,6,7,8,9,...,502,503,504,505,506,507,508,509,510,511
0,2.527940,0.262937,1.968569,1.235768,1.063264,0.347380,0.408746,0.475601,0.732204,1.524332,...,0.807808,1.064566,1.524986,3.107538,1.295686,0.707745,2.526153,0.500743,0.357597,0.454279
1,1.584208,0.197849,2.522341,1.361687,1.078774,0.385592,0.728359,0.675230,0.384694,1.338666,...,0.643229,1.251295,1.486058,3.455356,1.021523,0.689682,3.098471,0.358640,0.207957,0.484319
2,1.785293,0.128200,1.980437,0.783578,0.753184,0.352057,0.651084,0.890154,0.978148,1.841446,...,0.510674,1.283712,1.553033,3.458233,0.863063,0.775934,2.379142,0.465687,1.146188,0.990077
3,1.959851,0.204582,2.015643,1.334890,1.150177,0.510302,0.491724,0.398924,0.892903,1.082712,...,0.814672,0.816291,1.670235,3.052682,1.195130,0.716468,2.301680,0.586289,0.701486,0.790027
4,1.657544,0.195104,2.643000,1.287408,1.198735,0.445834,0.661746,0.788125,0.179200,1.506838,...,0.638161,1.220670,2.027056,3.302585,0.713268,0.582237,3.437111,0.311480,0.271206,0.409560
